#### Basic login website


>> Request: (hw1_main.py, login.html, math.html)
>>
>> Please use the flask web server platform to design your first web service with python, which has following       functions
>>
>> 1. login your system with your login name and password.
>>
>> 2. design one text box, so that user can put some data into the text box, and upload to the backend web service. .
>>
>> 3. calculate the maximum number, minimum number, Mean, SD etc.
>
>> Install Package:
>>
>> pip install flask
>>
>> pip install numpy

In [ ]:
# This is a login website
# Put on Cloud Server and Login
from flask import Flask, request, render_template
import numpy as np
app = Flask(__name__)
@app.route("/", methods=['GET','POST'])
def Login():
    if request.method == "POST":
        account=request.values["Account"]
        password=request.values["Password"]
        if account == "A000" and password == "A123":
            return render_template("math.html")
        else:
            return render_template("login.html", mes="Error Account")
    return render_template("login.html")
@app.route("/Calmath", methods=['POST'])
def CalMath():
    try:
        data = request.values["numbers"]
        num = list(map(int, data.split(' ')))
        max_num = max(num)
        min_num = min(num)
        mean_num = sum(num)/len(num)
        SD_num = np.std(num)
        return render_template("math.html",**locals())
    except ValueError:
        return render_template("math.html")

if __name__ == "__main__":
    app.run(port=5000)

#### AWS Practice

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import io, base64

from flask import Response
from flask import Flask, render_template, request, session, redirect
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from pyspark.sql import SparkSession


def loadDf():
    sc = SparkSession.builder.getOrCreate()
    spark = SparkSession(sc)
    df = spark.read.format("csv").option("encoding","BIG5").option("header", "true").option("inferSchema", "true").load("s3://s711083115/final.csv")
    return df
def getTitleArray(df,dataName):
    df = loadDf()
    df1 = df.select(dataName).distinct().sort(dataName)
    dfp1 = df1.toPandas()
    my_input=[str(xx) for xx in dfp1[dataName]]
    return my_input

app = Flask(__name__)
@app.route('/', methods=['GET'])
def initHTML():
    df = loadDf()
    my_years=getTitleArray(df,'癌症診斷年')
    my_years.insert(0,'All')
    my_place=getTitleArray(df,'縣市別')
    my_place.remove('全國')
    my_place.insert(0,'全國')
    return render_template('mainPage.html',my_place=my_place,my_years=my_years)

@app.route('/Result', methods=['GET'])
def plot_png():
    df = loadDf()
    year = request.args.get('Years')
    place = request.args.get('my_place')
    img = io.BytesIO()
    plt.switch_backend('agg')
    plt.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']
    if year == "All" :
        dataframe = df.filter(df["癌症別"]!="全癌症").filter(df["縣市別"]==place).filter(df["性別"]=="全")

        dataTotal = df.filter(df["癌症別"]!="全癌症").filter(df["縣市別"]==place).filter(df["性別"]=="全").select("癌症別","癌症發生數").groupBy("癌症別").sum("癌症發生數").sort('sum(癌症發生數)')
        dataTotal = dataframe.toPandas().tail(10)
        dds = dataframe.toPandas().head(10)
                
        describe = dataframe.select("癌症發生數","平均年齡","年齡中位數","粗率 (每10萬人口)").toPandas().describe()
        #cancer = dataframe.select("癌症別").toPandas()
        cancer = getTitleArray(df,"癌症別")
        plt.figure(figsize=(10,8),dpi=70)
        #ax = plt.subplot(111)
        for c in cancer:
            print("emptyemptyemptyempty ",c)
            for d in dataTotal["癌症別"]:
                if d == c:
                    df = dataframe.filter(dataframe["癌症別"]==c).select("癌症診斷年", "粗率 (每10萬人口)").groupBy("癌症診斷年").sum("粗率 (每10萬人口)").sort('癌症診斷年').toPandas()
                    x = df["癌症診斷年"]
                    y = df["sum(粗率 (每10萬人口))"]
                    plt.plot(x,y, label=c)
        plot_title = "各年度前十名癌症粗率比較"
        plt.xlabel("year")
        plt.ylabel("rate")
        plt.legend(loc='upper left')
        axes = plt.gca()
        axes.xaxis.label.set_size(18)
        axes.yaxis.label.set_size(18)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.grid()
        plt.savefig(img, format='png')
        plot_url = base64.b64encode(img.getvalue()).decode()
        return render_template('result.html',
                                tabledf=[dds.to_html(classes='data')],
                                tabledss={"Total": describe.to_html(classes='data')},
                                plot_title = plot_title,
                                plot_url = plot_url)
    else:
        dataframe = df.filter(df["癌症診斷年"]==year).filter(df["縣市別"]==place).filter(df["性別"]!="全").sort(df["癌症發生數"].desc())
        dds = dataframe.toPandas().head(10)
        print(dds)
        male_df = dataframe.filter(dataframe["性別"]=="男").select("癌症發生數","平均年齡","年齡中位數","粗率 (每10萬人口)").toPandas().describe()
        female_df = dataframe.filter(dataframe["性別"]=="女").select("癌症發生數","平均年齡","年齡中位數","粗率 (每10萬人口)").toPandas().describe()
        # x = 癌症發生數
        plt_df = dataframe.filter(dataframe["性別"]=="男").filter(dataframe["癌症別"]!="全癌症").select("癌症別","癌症發生數").sort(df["癌症發生數"].desc())
        x = plt_df.toPandas()["癌症發生數"]
        x_label = plt_df.toPandas()["癌症別"].head(5)
        pltf_df = dataframe.filter(dataframe["性別"]=="女").filter(dataframe["癌症別"]!="全癌症").select("癌症別","癌症發生數").sort(df["癌症發生數"].desc())
        y = pltf_df.toPandas()["癌症發生數"]
        y_label = pltf_df.toPandas()["癌症別"].head(5)

        plot_title = "Pie Graph"
        plt.figure(figsize=(20,8),dpi=70)
        f, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=False, sharey=False)
        ax1.set_title('Male',pad=0.4)
        ax2.set_title('Female', pad=0.4)
        ax1.pie(x, radius=1.2)
        ax2.pie(y, radius=1.2)
        ax1.legend(x_label, loc="center left", bbox_to_anchor=(1.2,0.5))
        ax2.legend(y_label, loc="center left", bbox_to_anchor=(1.2,0.5))
        plt.tight_layout(pad=0.4, w_pad=0.3, h_pad=0.3) 
        plt.savefig(img, format='png')
        plot_url = base64.b64encode(img.getvalue()).decode()
        return render_template('result.html',
                                tabledf=[dds.to_html(classes='data')],
                                tabledss={"Male": male_df.to_html(classes='data'),"Female":female_df.to_html(classes='data')},
                                plot_title = plot_title,
                                plot_url = plot_url)

if __name__ == "__main__":
    app.debug = True
    app.run(host='127.0.0.1', port=5000)
    